
# <center>  Proyecto Final - Fase 3  </center>

## Descripción
- En esta sección utilizaremos el preprocesamiento realizado en la Fase 3 para entrenar nuevos algoritmos.
- Utilizaremos los algoritmos de Regresión logística y Random Forest para clasificación.
  


## Problema de negocio

    
Un problema imperante en el entorno bancario a nivel global se enfoca en el riesgo crediticio o riesgo de impago. 
  
Este aspecto hace refencia a las pérdidas que incurren los bancos dado que los prestamistas son incapaces de devolver los créditos otorgados. 

Lo cual conlleva la siguiente pregunta: **¿Cómo poder reducir el número de personas que caen en mora crediticia?** 
  
Aspecto que se aborda a continuación.

## Objetivos del proyecto


El objetivo de este proyecto es construir un modelo que los prestatarios puedan usar para ayudar a tomar las mejores decisiones financieras.

Mediante el uso de diferentes algoritmos se obtendrá la probabilidad de impago en cada individuo. El principal indicador que se va usar para ver la bonanza de los distintos modelos va a ser el AUC (Area Under the Curve) generado por las curvas ROC.

## Importar librerías y verificar versiones

In [0]:
import sys
import numpy as np
import pandas as pd
import sklearn as sk
# Librerías para visualización
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

In [0]:
print('** Versiones Actuales | Requeridas **')
print('Python:\nVersion Actual:', sys.version, ' | Requerida >= 3.6')
print('NumPy:\nVersion Actual: {:10} | Requerida >= 1.16.2'.format(np.__version__))
print('Pandas:\nVersion Actual: {:10}| Requerida >= 0.24.2'.format(pd.__version__))
print('Scikit-learn:\nVersion Actual: {:10}| Requerida >= 1.2.1'.format(sk.__version__))
print('Matplotlib:\nVersion Actual: {:10} | Requerida >= 3.0.3'.format(matplotlib.__version__))
print('Seaborn:\nVersion Actual: {:10} |Requerida >= 0.9.0 '.format(sns.__version__))

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
import matplotlib.pyplot as plt

## Descripción de las variables contenidas en el _dataset_


El _dataset_ usado proviene de la plataforma **Kaggle**, y se encuentra separado en dos partes. 

La primera parte contiene los datos que serán usados para el entrenamiento del modelo, que serán nombrados como *datos_training*. La segunda parte son los datos que se usarán para el test del modelo *datos_test*. Estas muestras contienen 150.000 y 101.503 registros, respectivamente (aunque existen algunos campos que tienen distintos valores ausentes). 

Accesso a la Competencia **Give Me Some Credit:** [Link de acceso](https://www.kaggle.com/c/GiveMeSomeCredit/overview)

Link de descarga correspondiente a cada dataset: 
* [training data](https://storage.googleapis.com/datasets-academy/Track%20Data%20Science/Proyecto%20Final/cs-training.csv)
* [test data](https://storage.googleapis.com/datasets-academy/Track%20Data%20Science/Proyecto%20Final/cs-test.csv)

* **SeriousDlqin2yrs:** Persona que tiene 90 días de morosidad vencida o peor (Si = 1 y  No = 0)
* **RevolvingUtilizationOfUnsecuredLines:** El saldo total en tarjetas de crédito y líneas de crédito personales (excepto bienes inmuebles y sin deuda a plazos) sobre la suma de los límites de crédito
* **Age:** Edad prestamista en años 
* **NumberOfTime30-59DaysPastDueNotWorse:** Número de veces que el prestatario ha caido en mora entre 30 y 59 días, pero no ha empeorado en los últimos 2 años 
* **DebtRatio:** Pagos mensuales de deuda sobre el ingreso bruto mensual 
* **MonthlyIncome:** Ingreso mensual 
* **NumberOfOpenCreditLinesAndLoans:** Número de préstamos abiertos (cuotas como préstamos para automóviles o hipotecas) y líneas de crédito 
* **NumberOfTimes90DaysLate:** Número de veces que el prestatario ha caido en mora entre 90 días o más 
* **NumberRealEstateLoansOrLines:** Número de préstamos hipotecarios e inmobiliarios, incluidas las líneas de crédito hipotecario 
* **NumberOfTime60-89DaysPastDueNotWorse:** Número de veces que el prestatario ha caido en mora entre 60 y 89 días, pero no ha empeorado en los últimos 2 años 
* **NumberOfDependents:** Número de dependientes en la familia excluyendose a si mismos (ej: cónyuge, hijos, etc.)

## Carga de los datos

In [0]:
# Colocar la ruta del directorio correspondiente a cada dataset
ruta_training = 'https://storage.googleapis.com/datasets-academy/Track%20Data%20Science/Proyecto%20Final/cs-training.csv'
ruta_test     = 'https://storage.googleapis.com/datasets-academy/Track%20Data%20Science/Proyecto%20Final/cs-test.csv'

In [0]:
# Lectura de los datos usando pandas (creación de DataFrames)
datos_training = pd.read_csv(ruta_training, sep = ',', decimal = '.', index_col = 0)
datos_test     = pd.read_csv(ruta_test,     sep = ',', decimal = '.', index_col = 0)

In [0]:
# Imprimir el número de filas y columnas de ambos datasets
print('Filas y columnas - datos de entrenamiento', datos_training.shape)
print('Filas y columnas - datos de test', datos_test.shape)

In [0]:
# Imprimir los tipos de datos de cada columna en el dataset de entrenamiento
print(datos_training.dtypes)

## Ingeniería de variables

### Selección de variable objetivo y predictores del DataFrame `datos_training`

In [0]:
# En la variable y colocar la variable objetivo. En la variable X los predictores
y = datos_training.loc[:,'SeriousDlqin2yrs']
X = datos_training.loc[:, [i for i in datos_training.columns if i != 'SeriousDlqin2yrs']]

### División en datos de training y test

Para dividir la muestra de entrenamiento en dos submuestras de _training_ y _test_, se decide guardar una proporción de 80% _training_ 20% _test_. 

En los argumentos de `train_test_split` emplear el muestreo estratificado `stratify=y` y  `random_state=0`

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

In [0]:
# Imprimir las filas y columnas de los datos de training y test: para los predictores y para la variables objetivo
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

### Escalamiento (variables numéricas)

In [0]:
# Selección de las etiquetas de las variables numéricas
X_num = X.select_dtypes(include=np.number).columns
print(X_num)

### Pipeline - Ingeniería de variable

In [0]:
# Creación de un Pipeline para el preprocesamiento de las variables numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',  MinMaxScaler(feature_range=(0, 1)))])

In [0]:
# Delimitar el preprocesamiento de variables
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, X_num)])

### Modelos de Línea Base
A continuación, trabajaremos con el algoritmo de Random Forest para una tarea de clasificación. Emplearemos la Regresión Logística entrenada a manera de guía.

**1. Regresión Logística:** Crear en la variable `reg_log` el algoritmo de Regresión Logística con los siguientes argumentos: `max_iter=1000, class_weight='balanced', random_state=0, n_jobs = -1`

In [0]:
# Delimitar la configuración del algoritmo de Regresión Logística
reg_log = LogisticRegression(max_iter=1000,
                             class_weight='balanced', 
                             random_state=0,
                             n_jobs = -1)

**2. Random Forest:** Crear en la variable `rf` el algoritmo de Random Forest con los siguientes argumentos: `criterion='gini', class_weight='balanced', random_state=0, n_jobs = -1`

In [0]:
# Delimitar la configuración del algoritmo de Random Forest


### Pipeline - Ingeniería de variable y Algoritmo
Con el algoritmo de base creado en la sección anterior, construiremos un pipeline que agrupe la etapa de ingeniería de variables y algoritmo de Random Forest. Tomaremos la Regresión Logística como ejemplo.

In [0]:
# Creción de Pipeline regresión logística
reg_log_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', reg_log)])

reg_log_pipeline.fit(X_train, y_train)

In [0]:
# Creción de Pipeline Random Forest denominado rf_pipeline


### Cross-Validation
Utilizaremos los pipelines de la sección anterior para aplicar validación cruzada sobre el parámetro **max_depth** en el algoritmo de Random Forest. Utilizaremos la Regresión Logística a manera de ejemplo.

Proceso de Validación Cruzada para Random Forest:
1. Crear el grid de valores llamado `param_grid_rf` con el espacio de búsqueda apropiado (4 valores por el hiperparámetro)
2. En la variable `grid_rf` asignar `GridSearchCV` con los siguientes argumentos: `cv = 10, param_grid = param_grid_rf, estimator = rf_pipeline, scoring = 'roc_auc', n_jobs = -1`
3. Realizar el ajuste con respecto a los datos de entrenamiento
4. Imprimir el hiperparámetro óptimo y el mejor score para el algoritmo

In [0]:
# Validación Cruzada regresión logística
param_grid_reg_logistica = {'classifier__C': [1, 10, 100, 1000, 10000]}

grid_reg_logistica = GridSearchCV(cv = 10, 
                                  param_grid = param_grid_reg_logistica, 
                                  estimator = reg_log_pipeline, 
                                  scoring = 'roc_auc',
                                  n_jobs = -1)

grid_reg_logistica.fit(X_train, y_train)

# Imprimir el hiperparámetro óptimo y el mejor score
print(grid_reg_logistica.best_params_)
print(grid_reg_logistica.best_score_)

In [0]:
# Validación Cruzada Random Forest


### Algoritmo Optimizado
Con los parámetros óptimos encontrados en la sección anterior, construiremos un nuevo pipeline con la ingeniería de variables y la versión optimizada del algoritmo. Utilizaremos la Regresión Logística a manera de ejemplo.

In [0]:
# Regresión Logística optimizada
reg_log_opt = LogisticRegression(multi_class = 'ovr', 
                                 solver = 'lbfgs',
                                 C = grid_reg_logistica.best_params_['classifier__C'],
                                 max_iter = 1000,
                                 class_weight = 'balanced', 
                                 random_state = 0,
                                 n_jobs = -1)

# Creación del Pipeline para el preprocesamiento de los datos y ejecución del algortimo optimizado
reg_log_pipeline_opt = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', reg_log_opt)])

reg_log_pipeline_opt.fit(X_train, y_train)

# Clasificación de clientes (cut-off 0.5)
yHat_test_lg = reg_log_pipeline_opt.predict(X_test)

In [0]:
# Random Forest optimizado


Imprimir la métrica del AUC para analizar el desempeño del algoritmo en los datos de test. Utilizaremos la Regresión Logística a manera de ejemplo.

In [0]:
# AUC para regresión logística
scores_rl = reg_log_pipeline_opt.predict_proba(X_test)[:,1]
fpr_rl, tpr_rl, threshold_rl = metrics.roc_curve(y_test, scores_rl)

# Calculo métrica ROC-AUC
aucROC_rl = metrics.auc(fpr_rl, tpr_rl)
print(aucROC_rl)

In [0]:
# AUC para Random Forest


### Entrega
¿Cuál de los anteriores algoritmos escogería para entregar las predicciones al concurso de Kaggle? ¿Por qué?

In [0]:
#Su respuesta aquí


In [0]:
X_datos_test = datos_test.loc[:, [i for i in datos_test.columns if i != 'SeriousDlqin2yrs']]
X_datos_test.shape

In [0]:
prob_default = #Su código aquí
prob_default

In [0]:
df_entrega = pd.DataFrame(data=prob_default, 
                          columns = ["Probability"])

df_entrega['Id'] = range(1, prob_default.shape[0]+1)
df_entrega = df_entrega[["Id", "Probability"]] 

In [0]:
display(df_entrega)